# Bargain Bot

Goal is to make a bot that can perform bargain with a real human and make deals. There are couple of things here:

 - Understanding human's input and get context out of it.
 - If the context matches the goal, see if it can fulfill bot's goal in short or long term.
 - Learn the process of bargaining
 - Generate the answer back to human
 - It the input is out of context, try and give a relevant answer.

### Experiment 1: Understanding
#### I'll give you four bananas if you give me two mangos.

In [3]:
input = "I'll give you four bananas if you give me two mangos."

In [35]:
# setting up the environment
file_names=['gs', 'gswin32c.exe', 'gswin64c.exe']
%xmode verbose

import nltk
from nltk.stem.snowball import SnowballStemmer


Exception reporting mode: Verbose


In [ ]:
tokens = nltk.word_tokenize(input)


tokens

In [17]:
tagged = nltk.pos_tag(tokens)
tagged

[('I', 'PRP'),
 ("'ll", 'MD'),
 ('give', 'VB'),
 ('you', 'PRP'),
 ('four', 'CD'),
 ('bananas', 'NNS'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('give', 'VBP'),
 ('me', 'PRP'),
 ('two', 'CD'),
 ('mangos', 'NNS'),
 ('.', '.')]

In [19]:
entities = nltk.chunk.ne_chunk(tagged)

print ( entities.__repr__())

Tree('S', [('I', 'PRP'), ("'ll", 'MD'), ('give', 'VB'), ('you', 'PRP'), ('four', 'CD'), ('bananas', 'NNS'), ('if', 'IN'), ('you', 'PRP'), ('give', 'VBP'), ('me', 'PRP'), ('two', 'CD'), ('mangos', 'NNS'), ('.', '.')])


In [22]:
# defining the objects the bot has and how much fuck it gives about it. 
objects = {"mango": [10, 1], "banana": [10, 2]}

In [109]:
## Lets start with easier statements and approach
sentence1 = "I'll give you two bananas"
sentence2 = "you give me three mangos"

stemmer = SnowballStemmer("english")

numbers = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "ten" : 10}

def getContext(tree):
    global numbers
    situation = None
    entity = None
    count = None
    for word, wordType in tree:
        word = stemmer.stem(word)
        if (wordType == 'NNS'): 
            if word in objects:
                entity = word
            else:
                entity = None
                break

        if (wordType == 'PRP' and word == 'you'): situation = "ADD"
        elif(wordType == 'PRP' and word == 'me'): situation = "SUB"

        if (wordType == 'CD' and word in numbers):
            count = numbers[word]
        elif (wordType == 'CD'):
            count = None
            break
    return [situation, entity, count]
        
tree1 = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence1.lower())))
print (tree1.__repr__())
situation, entity, count = getContext(tree1)
print (situation, entity, count)

tree2 = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence2.lower())))
print (tree1.__repr__())
situation, entity, count = getContext(tree2)
print (situation, entity, count)

Tree('S', [('i', 'NN'), ("'ll", 'MD'), ('give', 'VB'), ('you', 'PRP'), ('two', 'CD'), ('bananas', 'NNS')])
ADD banana 2
Tree('S', [('i', 'NN'), ("'ll", 'MD'), ('give', 'VB'), ('you', 'PRP'), ('two', 'CD'), ('bananas', 'NNS')])
SUB mango 3


In [110]:
def act(situation, entity, count):
    global objects
    if situation == "ADD":
#         print (entity, objects[entity])
        objects[entity][0] += count
        print ("Thanks for %d %ss" % (count, entity))
    if situation == "SUB":
        if objects[entity][0] < count:
            if objects[entity][0] == 0:
                print ("I have no %ss. Can't give you %d" % (entity, count))
            else:
                print ("I only have %d %ss. Can't give you %d" % (objects[entity][0], entity, count))
            return
        objects[entity][0] -= count
        print ("Giving you %d %ss" % (count, entity))
        
    print ("Now I have:")
    for k, v in objects.items():
        print ("%s %ss" % (v[0], k))

In [111]:
print (sentence1.lower())
tree1 = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence1.lower())))
situation, entity, count = getContext(tree1)
act (situation, entity, count)

i'll give you two bananas
Thanks for 2 bananas
Now I have:
18 mangos
21 bananas


In [112]:
print (sentence2.lower())
tree2 = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence2.lower())))
situation, entity, count = getContext(tree2)
act (situation, entity, count)

you give me three mangos
Giving you 3 mangos
Now I have:
15 mangos
21 bananas


In [113]:
def actOnSentense(sentence):
    print ("ASK: ", sentence)
    print ("ANSWER:")
    tree = nltk.chunk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence.lower())))
    situation, entity, count = (getContext(tree))
    if (situation == None):
        return print ("I dont understand the situation")
    if (entity == None):
        return print ("I don't have the knowledge about the entity you asking")
    if (count == None):
        return print ("Didn't get the count right")
    
    act (situation, entity, count)
    

In [130]:
# let's try one simple example
actOnSentense("give me two mangos")

ASK:  give me two mangos
ANSWER:
Giving you 2 mangos
Now I have:
38 mangos
5 bananas


In [131]:
actOnSentense("fine I'll give you three mangos")
actOnSentense("fine I'll give you three mangos")
actOnSentense("fine I'll give you three mangos")
actOnSentense("give me ten bananas")
actOnSentense("give me three bananas")
actOnSentense("give me three bananas")
actOnSentense("give me three bananas")
actOnSentense("give me five bananas")
actOnSentense("giving you five bananas")
actOnSentense("giving you five mangos")
actOnSentense("give me five mango")

ASK:  fine I'll give you three mangos
ANSWER:
Thanks for 3 mangos
Now I have:
41 mangos
5 bananas
ASK:  fine I'll give you three mangos
ANSWER:
Thanks for 3 mangos
Now I have:
44 mangos
5 bananas
ASK:  fine I'll give you three mangos
ANSWER:
Thanks for 3 mangos
Now I have:
47 mangos
5 bananas
ASK:  give me ten bananas
ANSWER:
Didn't get the count right
ASK:  give me three bananas
ANSWER:
Giving you 3 bananas
Now I have:
47 mangos
2 bananas
ASK:  give me three bananas
ANSWER:
I only have 2 bananas. Can't give you 3
ASK:  give me three bananas
ANSWER:
I only have 2 bananas. Can't give you 3
ASK:  give me five bananas
ANSWER:
I only have 2 bananas. Can't give you 5
ASK:  giving you five bananas
ANSWER:
Thanks for 5 bananas
Now I have:
47 mangos
7 bananas
ASK:  giving you five mangos
ANSWER:
Thanks for 5 mangos
Now I have:
52 mangos
7 bananas
ASK:  give me five mango
ANSWER:
Giving you 5 mangos
Now I have:
47 mangos
7 bananas
